# Parcial 1
## Mecánica Cuántica
### Thomas Martinod y Fernando Londoño

El desarrollo del parcial requiere de muchos cálculos propios del álgebra lineal. Para estos cálculos se empleará la librería **SymPy**, librería de cálculo simbólico que permite desarrollar el parcial sin error numérico. A cambio de este desarrollo simbólico, las respuestas de algunos numerales tendrán formas irreducibles pero largas, además de que el código tiene requiere de un gran tiempo de cómputo para muchas matrices de entrada.

In [1]:
# Se importa sympy para todo el parcial
from sympy import *
from sympy.physics.quantum import Dagger

# Se define la unidad imaginaria como i minúscula por comodidad
i = I
display(i)

I

## Problema 1

Se definen los operadores $H$ y $L$ como matrices simbólicas de SymPy. Los operadores $H$ y $L$ definidos fueron los siguientes:

\begin{equation}
H = \left[\begin{matrix}0 & 0 & i\\0 & 3 & 0\\- i & 0 & 0\end{matrix}\right], \quad L = \left[\begin{matrix}\pi & i \pi & 0\\- i \pi & 2 \pi & \pi\\0 & \pi & 3 \pi\end{matrix}\right]
\end{equation}



In [2]:
# Definición de la matriz H
H = Matrix([[0,0,i],[0,3,0],[-i,0,0]])

# Definición de la matriz L
L = pi*Matrix([[1,i,0],[-i,2,1],[0,1,3]])

# Se enseñan ambas matrices
display(H,L)


Matrix([
[ 0, 0, I],
[ 0, 3, 0],
[-I, 0, 0]])

Matrix([
[   pi, I*pi,    0],
[-I*pi, 2*pi,   pi],
[    0,   pi, 3*pi]])

Para ver que son hermíticos, aplicamos la función `Dagger` de SymPy a cada matriz y vemos que son la misma que antes de aplicar el operador daga. Veamos, $H^\dagger = $

In [3]:
# Hermiticidad de H
display(Dagger(H))
print(Dagger(H) == H)

Matrix([
[ 0, 0, I],
[ 0, 3, 0],
[-I, 0, 0]])

True


Y $L^\dagger = $

In [4]:
# Hermiticidad de L
display(Dagger(L))
print(Dagger(L) == L)

Matrix([
[   pi, I*pi,    0],
[-I*pi, 2*pi,   pi],
[    0,   pi, 3*pi]])

True


## Problema 2
Se debe calcular el conmutador $[H,L]$. Para ello se define la función `Commutator`, que toma dos matrices $A $ y $ B$ de sympy y retorna el conmutador $[A,B]=AB-BA$.

In [5]:
# Se define la función commutator
def Commutator(A, B):
    # El conmutador de A y B simplificando lo máximo posible cada una de las entradas de la matriz.
    return (A*B - B*A).applyfunc(simplify)

# Se imprime el conmutador de las matrices H y L
display(Commutator(H,L))

Matrix([
[      0, -2*I*pi, 2*I*pi],
[-2*I*pi,       0,   2*pi],
[ 2*I*pi,   -2*pi,      0]])

## Problema 3

Para calacular los autovalores y autovectores de las matrices $H$, $L$ se crea la función `getEigen`, que recibe como entrada una matriz $M$ y retorna primero un vector columna con los autovalores de $M$ (si hay multiplicidad se repite en el vector el autovalor), y segundo **una matriz cuyas columnas son los autovectores de la matriz $M$**. Cabe aclarar que esta función emplea el método de SymPy `eigenvects` internamente.

In [6]:
def getEigen(M):
    # Obtiene el tamaño de la matriz M
    m = M.shape[0]
    # Se inicializan dos listas vacías para los autovalores (vals) y los autovectores (vects)
    vals, vects = [], []

    # Obtiene los valores propios y los vectores propios de la matriz M
    eig = M.eigenvects()

    # Itera sobre los valores propios, sus multiplicidades y los vectores propios correspondientes
    for i in eig:
        # Itera sobre el número de vectores propios asociados a un valor propio
        for j in range(i[1]):
            # Añade el valor propio expandido y simplificado a la lista de valores propios
            vals.append(simplify(i[0].expand(complex=True)))
            # Añade el vector propio expandido y simplificado a la lista de vectores propios
            vects.append(simplify(i[2][j].expand(complex=True)))

    # Crea una matriz de ceros del mismo tamaño que la lista de vectores propios
    Svects = zeros(m)

    # Convierte los elementos de la lista de vectores propios en objetos Matrix y los asigna a Svects
    for i in range(len(vects)):
        Svects[i] = Matrix(vects[i])

    # Devuelve una tupla que contiene una matriz m x 1 de valores propios y una matriz m x m de vectores propios
    return Matrix(vals), Svects

Ahora, habiendo definido el método, se calculan los autovalores y autovectores de $H$ y $L$:

In [7]:
# Llama a la función getEigenvectsEigenvals con la matriz Htoy como argumento
Hvals, Hvects = getEigen(H)

# Muestra los valores propios y los vectores propios
display(Hvals, Hvects)

Matrix([
[-1],
[ 1],
[ 3]])

Matrix([
[-I, I, 0],
[ 0, 0, 1],
[ 1, 1, 0]])

In [8]:
# Llama a la función getEigen con la matriz L como argumento
Lvals, Lvects = getEigen(L)

# Muestra los valores propios y los vectores propios
display(Lvals, Lvects)

Matrix([
[            2*pi],
[pi*(2 - sqrt(3))],
[pi*(sqrt(3) + 2)]])

Matrix([
[-I, sqrt(3)*I + 2*I, -sqrt(3)*I + 2*I],
[-1,    -sqrt(3) - 1,     -1 + sqrt(3)],
[ 1,               1,                1]])

Ahora, para obtener los autovalores **normalizados** de cada una de las matrices, se definen dos funciones. La función `getNorm` recibe un vector de Sympy y retorna su norma, calculada como $||\vec{v}|| = \sqrt{|v_1|^2 + \cdots |v_n|^2}$ (la función `Abs` se sympy retorna el módulo de un complejo). Mientras que la función `Normalize` llama la función `getNorm` y retorna el vector dividido  su norma (normalizado).

In [9]:
def getNorm(v):
    # Calcula la norma del vector v utilizando la suma de los cuadrados de sus componentes
    # y toma la raíz cuadrada del resultado. Luego simplifica la expresión.
    norm = simplify(sqrt(sum(Abs(i)**2 for i in v)).expand(complex=True))
    return norm

def Normalize(v):
    # Calcula la norma del vector v utilizando la función getNorm definida anteriormente.
    norm = getNorm(v)

    # Normaliza el vector v dividiendo cada componente por la norma.
    normalized_vector = simplify((v*(1/norm)).expand(complex=True))

    return normalized_vector

Simplemente falta llamar el método normalize para cada autovalor de $H$ y $L$, es decir, para cada columna de las matrices *Hvects* y *Lvects*. Para ello se escriben dos ciclos implícitos que guarden en otra matriz de sympy estos mismos autovalores normalizados.

In [10]:
# Normaliza los vectores propios de las matrices H y L
NHvects = Matrix([Normalize(Hvects.col(v)) for v in range(Hvects.shape[1])]).reshape(*Hvects.shape).T
NLvects = Matrix([Normalize(Lvects.col(v)) for v in range(Lvects.shape[1])]).reshape(*Lvects.shape).T

# Al final se transpone porque por defecto de almacenan vectores como filas

# Muestra los vectores propios normalizados de H y L
display(NHvects, NLvects)

print(latex(NLvects.col(2)))

Matrix([
[-sqrt(2)*I/2, sqrt(2)*I/2, 0],
[           0,           0, 1],
[   sqrt(2)/2,   sqrt(2)/2, 0]])

Matrix([
[-sqrt(3)*I/3,                      sqrt(-12 - 6*sqrt(3))/6,                      sqrt(-12 + 6*sqrt(3))/6],
[  -sqrt(3)/3, -sqrt(6)*(1 + sqrt(3))/(6*sqrt(sqrt(3) + 2)), sqrt(6)*(-1 + sqrt(3))/(6*sqrt(2 - sqrt(3)))],
[   sqrt(3)/3,                sqrt(6)/(6*sqrt(sqrt(3) + 2)),                sqrt(6)/(6*sqrt(2 - sqrt(3)))]])

\left[\begin{matrix}\frac{\sqrt{-12 + 6 \sqrt{3}}}{6}\\\frac{\sqrt{6} \left(-1 + \sqrt{3}\right)}{6 \sqrt{2 - \sqrt{3}}}\\\frac{\sqrt{6}}{6 \sqrt{2 - \sqrt{3}}}\end{matrix}\right]


Para verificar (si se desea) que los vectores propios están normalizados, se llama la función `getNorm` con alguna de las columnas de las matrices anteriores como entrada. Por ejemplo:

In [11]:
# Se enseña el tercer autovector normalizado de L
display(NLvects.col(2))

# Se enseña la norma de este vector
display(getNorm(NLvects.col(2)))

Matrix([
[                     sqrt(-12 + 6*sqrt(3))/6],
[sqrt(6)*(-1 + sqrt(3))/(6*sqrt(2 - sqrt(3)))],
[               sqrt(6)/(6*sqrt(2 - sqrt(3)))]])

1

Si además se desea visualizar de forma numérica alguno de los vectores, se utiliza el método `evalf(n)`de SymPy que evalua una expresión con los $n$ decimales que de requieran como entrada.

## Punto 6

En el numeral 3 se derivaron los conjuntos base formados por los autovectores normalizados de cada matriz. Como conjunto, las bases $B_H$ y $B_L$ son entonces:

\begin{equation}
    B_H = \left\{\left[\begin{matrix}- \frac{\sqrt{2} i}{2}\\0\\\frac{\sqrt{2}}{2}\end{matrix}\right], \left[\begin{matrix}\frac{\sqrt{2} i}{2}\\0\\\frac{\sqrt{2}}{2}\end{matrix}\right], \left[\begin{matrix}
        0\\ 1\\ 0
    \end{matrix}\right] \right\}, \quad B_L = \left\{\left[\begin{matrix}- \frac{\sqrt{3} i}{3}\\- \frac{\sqrt{3}}{3}\\\frac{\sqrt{3}}{3}\end{matrix}\right], \left[\begin{matrix}\frac{\sqrt{-12 - 6 \sqrt{3}}}{6}\\- \frac{\sqrt{6} \left(1 + \sqrt{3}\right)}{6 \sqrt{\sqrt{3} + 2}}\\\frac{\sqrt{6}}{6 \sqrt{\sqrt{3} + 2}}\end{matrix}\right],\left[\begin{matrix}\frac{\sqrt{-12 + 6 \sqrt{3}}}{6}\\\frac{\sqrt{6} \left(-1 + \sqrt{3}\right)}{6 \sqrt{2 - \sqrt{3}}}\\\frac{\sqrt{6}}{6 \sqrt{2 - \sqrt{3}}}\end{matrix}\right] \right\}
\end{equation}

Para demostrar que en efecto estos dos conjuntos representan **bases ortonormales** para el espacio $\mathbb{C}^3$, se debe probar la completez y ortonormalidad de cada conjunto. Se define inicialmente la función `InnerProd` que toma dos vectores $v,w$ y retorna $\braket{v|w} = v^\dagger w$.

In [12]:
def InnerProd(v, w):
    # (Dagger(v)*w) realiza la multiplicación matricial del conjugado transpuesto de v con w.
    # [0] extrae el primer elemento del resultado de la multiplicación, ya que el resultado es una matriz de una sola fila y columna.
    return ((Dagger(v) * w)[0].evalf())


Ahora, para verificar la **ortonormalidad** de cada conjunto, se crea la función `ONverification`, que imprime cada una de las combinaciones de los productos internos de las columnas de una matriz de entrada $M$.

In [13]:
def ON(M):
    for i in range(M.shape[1]):
        for j in range(M.shape[1]):
            print('El producto interno del vector ' + str(i+1) + ' con el vector ' + str(j+1) + ' es: ' + str(InnerProd(M.col(i), M.col(j))))

Como las matrices *NHvects* y *NLvects* tienen en sus columnas los vectores de sus respectivas bases, simplemente se llama a la función anterior con cada matriz.

In [14]:
# Verificación de la ortonormalidad de H
ON(NHvects)

El producto interno del vector 1 con el vector 1 es: 1.00000000000000
El producto interno del vector 1 con el vector 2 es: 0
El producto interno del vector 1 con el vector 3 es: 0
El producto interno del vector 2 con el vector 1 es: 0
El producto interno del vector 2 con el vector 2 es: 1.00000000000000
El producto interno del vector 2 con el vector 3 es: 0
El producto interno del vector 3 con el vector 1 es: 0
El producto interno del vector 3 con el vector 2 es: 0
El producto interno del vector 3 con el vector 3 es: 1.00000000000000


In [15]:
# Verificación de la ortonormalidad de L
ON(NLvects)

El producto interno del vector 1 con el vector 1 es: 1.00000000000000
El producto interno del vector 1 con el vector 2 es: 0.e-125
El producto interno del vector 1 con el vector 3 es: 0.e-125
El producto interno del vector 2 con el vector 1 es: 0.e-125
El producto interno del vector 2 con el vector 2 es: 1.00000000000000
El producto interno del vector 2 con el vector 3 es: 0.e-125
El producto interno del vector 3 con el vector 1 es: 0.e-125
El producto interno del vector 3 con el vector 2 es: 0.e-125
El producto interno del vector 3 con el vector 3 es: 1.00000000000000


Claramente la evaluación en punto flotante de las expresiones radicales de los autovectores induce un mínimo error numérico. No obastante, es del orden de $10^{-125}$, luego, se puede afirmar que ambas bases son ortonormales.

Por otro lado, se debe verificar la completez de cada una de las bases. Esta demostración es equivalente a enseñar que la suma de todos los **proyectores** de la base suman a la matriz identidad. Es decir, para $H$ se debe ver que:

\begin{equation}
    \sum_{i=1}^3 \ket{h_i}\bra{h_i} = \mathcal{I}_3
\end{equation}

al igual que para $L$.

El primer paso es crear un método que retorne los proyectores de un conjunto (matriz) de vectores base. La función `getProjectors` hace justo eso, computando el ket-bra para cada columna de una matriz de entrada.

In [16]:
def getProjectors(M):
    # Retorna una lista con los proyectores en sus componentes (en el orden de las columnas de la matriz)
    return [M.col(i) * Dagger(M.col(i)) for i in range(M.shape[1])]

Ahora, para hallar los proyectores de caada base, solo es cuestión de llamar este método con las matrices *Hvects* y *Lvects* y mostrar cada uno de los elementos de la lista resultante.

In [17]:
# Proyectores de NHvects (H)
ProyNH = getProjectors(NHvects)
display(ProyNH[0], ProyNH[1], ProyNH[2])

Matrix([
[1/2, 0, -I/2],
[  0, 0,    0],
[I/2, 0,  1/2]])

Matrix([
[ 1/2, 0, I/2],
[   0, 0,   0],
[-I/2, 0, 1/2]])

Matrix([
[0, 0, 0],
[0, 1, 0],
[0, 0, 0]])

In [18]:
# Proyectores de NLvects (L)
ProyNL = getProjectors(NLvects)
display(ProyNL[0], ProyNL[1], ProyNL[2])

Matrix([
[ 1/3,  I/3, -I/3],
[-I/3,  1/3, -1/3],
[ I/3, -1/3,  1/3]])

Matrix([
[                   -I*sqrt(-12 - 6*sqrt(3))*sqrt(6*sqrt(3) + 12)/36, -sqrt(6)*sqrt(-12 - 6*sqrt(3))*(1 + sqrt(3))/(36*sqrt(sqrt(3) + 2)), sqrt(6)*sqrt(-12 - 6*sqrt(3))/(36*sqrt(sqrt(3) + 2))],
[sqrt(6)*I*(1 + sqrt(3))*sqrt(6*sqrt(3) + 12)/(36*sqrt(sqrt(3) + 2)),                                  (1 + sqrt(3))**2/(6*(sqrt(3) + 2)),                     -(1 + sqrt(3))/(6*(sqrt(3) + 2))],
[             -sqrt(6)*I*sqrt(6*sqrt(3) + 12)/(36*sqrt(sqrt(3) + 2)),                                    -(1 + sqrt(3))/(6*(sqrt(3) + 2)),                                  1/(6*(sqrt(3) + 2))]])

Matrix([
[                     -I*sqrt(-12 + 6*sqrt(3))*sqrt(12 - 6*sqrt(3))/36, sqrt(6)*sqrt(-12 + 6*sqrt(3))*(-1 + sqrt(3))/(36*sqrt(2 - sqrt(3))), sqrt(6)*sqrt(-12 + 6*sqrt(3))/(36*sqrt(2 - sqrt(3)))],
[-sqrt(6)*I*(-1 + sqrt(3))*sqrt(12 - 6*sqrt(3))/(36*sqrt(2 - sqrt(3))),                                 (-1 + sqrt(3))**2/(6*(2 - sqrt(3))),                     (-1 + sqrt(3))/(6*(2 - sqrt(3)))],
[               -sqrt(6)*I*sqrt(12 - 6*sqrt(3))/(36*sqrt(2 - sqrt(3))),                                    (-1 + sqrt(3))/(6*(2 - sqrt(3))),                                  1/(6*(2 - sqrt(3)))]])

Para comprender mejor los proyectores de los autokets normalizados de $L$, evaluemos numéricamente los últimos dos:

In [19]:
display(ProyNL[0], ProyNL[1].evalf(3), ProyNL[2].evalf(3))

Matrix([
[ 1/3,  I/3, -I/3],
[-I/3,  1/3, -1/3],
[ I/3, -1/3,  1/3]])

Matrix([
[   0.622, -0.455*I, 0.167*I],
[ 0.455*I,    0.333,  -0.122],
[-0.167*I,   -0.122,  0.0447]])

Matrix([
[  0.0447, 0.122*I, 0.167*I],
[-0.122*I,   0.333,   0.455],
[-0.167*I,   0.455,   0.622]])

Ahora, para verificar la completez, falta hacer la suma de los proyectores de $B_H$ y $B_L$. Sumando para $B_H$ los proyectores obtenidos:

In [20]:
# Suma de proyectores de la base H
display(ProyNH[0] + ProyNH[1] + ProyNH[2])

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

In [21]:
# Suma de proyectores de la base L
display(simplify((ProyNL[0] + ProyNL[1] + ProyNL[2]).expand(complex=True)))

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

Entonces, como en ambos casos se obtuvo la matriz identidad de orden 3, se puede afirmar que $B_H$ y $B_L$ son bases completas ortonormales para $\mathbb{C}^3$.

## Punto 7

En este punto de debe calcular la matriz de transformación $U_{H\rightarrow L} = U$ que reescriba cualquier vector en la base $B_H$ al respectivo vector en la base $B_L$. Aunque el desarrollo matemático se realiza de forma explícita en el documento *.pdf* adjunto a este código, lo mas importante de ver es que la matriz de transformción $U$ viene dada por el producto de la matriz de autovectores normaizados de $H$ por la invesa de la matriz de los autovectores normalizados de $L$. Es decir,  $U =$ *HNvects* $\times$ *LNvects*$^{-1}$.

Haciendo este producto:

In [22]:
U = simplify((NLvects.inv() * NHvects).expand(complex==True))

# Se muestra la matriz U
display(U)

Matrix([
[                           sqrt(6)/3,                                   0,                                        -sqrt(3)/3],
[-(sqrt(3) + 3)/(6*sqrt(sqrt(3) + 2)), (1 + sqrt(3))/(2*sqrt(sqrt(3) + 2)), -(5*sqrt(6) + 9*sqrt(2))/(6*(sqrt(3) + 2)**(3/2))],
[   sqrt(2 - sqrt(3))*(sqrt(3) + 3)/6,   (1 + sqrt(3))*sqrt(2 - sqrt(3))/2,         sqrt(2 - sqrt(3))*(sqrt(6) + 3*sqrt(2))/6]])

Ahora, se debe verificar que esta matriz es **unitaria**. Para ello, se calculan $U^{-1}$ y $U^\dagger$ y se verifica que $U^{-1} = U^\dagger \iff U^{-1} - U^\dagger = \mathcal{O}_3$.

In [23]:
# Se calcula la inversa de U
Uinv = simplify(U.inv())
display(Uinv)

# Se calcula la matriz adjunta de U
Udagger = Dagger(U)
display(Udagger)

Matrix([
[ sqrt(6)/3,         -sqrt(sqrt(3) + 2)/2 + sqrt(3)*sqrt(sqrt(3) + 2)/6,          (5*sqrt(3) + 9)/(6*sqrt(2 - sqrt(3))*(4*sqrt(3) + 7))],
[         0,    (56*sqrt(3) + 97)/((71 + 41*sqrt(3))*sqrt(sqrt(3) + 2)),          (4*sqrt(3) + 7)/(sqrt(2 - sqrt(3))*(19 + 11*sqrt(3)))],
[-sqrt(3)/3, -sqrt(2)*sqrt(sqrt(3) + 2)/2 + sqrt(6)*sqrt(sqrt(3) + 2)/6, (5*sqrt(2) + 3*sqrt(6))/(2*sqrt(2 - sqrt(3))*(12 + 7*sqrt(3)))]])

Matrix([
[ sqrt(6)/3,              -(sqrt(3) + 3)/(6*sqrt(sqrt(3) + 2)),         sqrt(2 - sqrt(3))*(sqrt(3) + 3)/6],
[         0,               (1 + sqrt(3))/(2*sqrt(sqrt(3) + 2)),         (1 + sqrt(3))*sqrt(2 - sqrt(3))/2],
[-sqrt(3)/3, -(5*sqrt(6) + 9*sqrt(2))/(6*(sqrt(3) + 2)**(3/2)), sqrt(2 - sqrt(3))*(sqrt(6) + 3*sqrt(2))/6]])

Y Aunque las dos matrices por inspección parezcan diferentes, será su resta quien verificará su hermiticidad:

In [24]:
display(simplify(Uinv - Udagger))

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

Ya habiendo verificado que $U^\dagger = U^{-1}$, esta claro que $U^\dagger U = \mathcal{I}_3$. Verificae este hecho es sencillo:

In [25]:
display(simplify(Udagger * U))

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

Ahora bien, para finalmente reescribir la matriz $H$ en términos de la base $B_L$, se calcula la transformación $H' = H_L = UHU^\dagger$:

In [26]:
H_L = simplify((U * H * Udagger).expand(complex=True))
H_L

Matrix([
[                                         0,                                                                                                                                                               -I*(5 + 3*sqrt(3))/(2*(sqrt(3) + 2)**(3/2)),                                                                                                                                                                                  sqrt(-2 + sqrt(3))*(1 + sqrt(3))/2],
[I*(5 + 3*sqrt(3))/(2*(sqrt(3) + 2)**(3/2)),                                                                                                                                                                                                       3/2, ((sqrt(3) + 2)**3*(9*sqrt(6 - 3*sqrt(3)) + 18*sqrt(2 - sqrt(3)) - 2*sqrt(-4 + 2*sqrt(3)) - sqrt(-12 + 6*sqrt(3)))/6 + 2*sqrt(6)*I*(sqrt(3) + 2)**(3/2)/3 + 7*sqrt(2)*I*(sqrt(3) + 2)**(3/2)/6)/(sqrt(3) + 2)**(7/2)],
[      -I*(1 + sqrt(3))*sqrt(2 - sqrt(3))/2, (84*sqrt(6 - 3*sqrt(3)

Y numéricamente, esta adquiere la forma:

In [27]:
H_L.evalf(3)

Matrix([
[       0,         -0.707*I,          0.707*I],
[ 0.707*I,              1.5, 1.5 - 3.84e-12*I],
[-0.707*I, 1.5 + 4.64e-12*I,              1.5]])

## Punto 8

En este punto se deben obtener el determinante y la traza de las matrices $H$ y $L$, adicionalmente se deben obtener el determinante y la traza nuevamente pero de las matrices en su forma diagonal.

Para esto, se utiliza la función `det` que obtiene el determinante de una matriz y también se define la función `getTrace` que retorna la traza de una matriz dada. También se usa la función `diagonalize` que da las matrices necesarias para la diagonalización $PDP^{-1} = U$.

In [28]:
# Calcula y muestra el determinante de la matriz H
detH = H.det()
print("El determinante de H es", detH)

# Calcula y muestra el determinante de la matriz L
detL = L.det()
print("El determinante de L es", detL)

# Función para calcular la traza de una matriz
def getTrace(H):
    tr = 0
    # Recorre la diagonal principal de la matriz y suma sus elementos
    for i in range(0, H.shape[1]):
      tr += H.col(i)[i]
    return tr

# Muestra la traza de las matrices H y L
print("La traza de H es", getTrace(H))
print("La traza de L es", getTrace(L))

# Diagonaliza la matriz H y muestra las matrices de vectores propios y valores propios
P, D = H.diagonalize()
print("Matriz P (vectores propios):")
display(P)
print("\nMatriz D (valores propios en la diagonal):")
display(D)

# Calcula la matriz diagonalizada de H y muestra su determinante y traza
HD = P * D * P.inv()
detHD = HD.det()
tzHD = getTrace(HD)
print("El determinante de H en su forma diagonal es", detHD)
print("La traza de H en su forma diagonal es", tzHD)

# Diagonaliza la matriz L y muestra las matrices de vectores propios y valores propios
P, D = L.diagonalize()
print("Matriz P (vectores propios):")
display(P)
print("\nMatriz D (valores propios en la diagonal):")
display(D)

# Calcula la matriz diagonalizada de L y muestra su determinante y traza
LD = P * D * P.inv()
detLD = LD.det()
tzLD = simplify(getTrace(LD))
print("El determinante de L en su forma diagonal es", detLD)
print("La traza de L en su forma diagonal es", tzLD)

El determinante de H es -3
El determinante de L es 2*pi**3
La traza de H es 3
La traza de L es 6*pi
Matriz P (vectores propios):


Matrix([
[-I, I, 0],
[ 0, 0, 1],
[ 1, 1, 0]])


Matriz D (valores propios en la diagonal):


Matrix([
[-1, 0, 0],
[ 0, 1, 0],
[ 0, 0, 3]])

El determinante de H en su forma diagonal es -3
La traza de H en su forma diagonal es 3
Matriz P (vectores propios):


Matrix([
[-I, sqrt(3)*I + 2*I, -sqrt(3)*I + 2*I],
[-1,    -sqrt(3) - 1,     -1 + sqrt(3)],
[ 1,               1,                1]])


Matriz D (valores propios en la diagonal):


Matrix([
[2*pi,                0,                0],
[   0, pi*(2 - sqrt(3)),                0],
[   0,                0, pi*(sqrt(3) + 2)]])

El determinante de L en su forma diagonal es 2*pi**3
La traza de L en su forma diagonal es 6*pi


## Punto 9


Este punto pide encontrar dos estados arbitrarios como combinación lineal de los vectores base de $B_H$ y $B_L$. Para esto se tomaron coeficientes 2, -1, 3 en el caso de los vectores base de $B_H$ y 3, 4, -2 en el caso de los vectores base de la matriz $B_L$.


In [29]:
# Se definen dos estados arbitrarios como combinación lineal de los vectores base de H y L. (phi1 está asociado a H y phi2 asociado a L)
phi1 = 2*NHvects.col(0) - 1*NHvects.col(1) + 3*NHvects.col(2)
phi2 = 3*NLvects.col(0) + 4*NLvects.col(1) - 2*NLvects.col(1)

# Se muestra por pantalla el resultado de los estados arbitrarios phi1 y phi2
display(phi1)
display(phi2)

Matrix([
[-3*sqrt(2)*I/2],
[             3],
[     sqrt(2)/2]])

Matrix([
[                  -sqrt(3)*I + sqrt(-12 - 6*sqrt(3))/3],
[-sqrt(3) - sqrt(6)*(1 + sqrt(3))/(3*sqrt(sqrt(3) + 2))],
[               sqrt(6)/(3*sqrt(sqrt(3) + 2)) + sqrt(3)]])

## Punto 10


Este punto pide determinar el producto de las incertidumbres asociadas a $H$y $L$ evaluados en los dos estados arbitrarios ($\ket{\phi_1}$ y $\ket{\phi_2}$) definidos en el punto anterior.

Para ello, se define la incertidumbre al cuadrado de $H$ y $L$ en el estado $\ket{\phi_1}$ y $\ket{\phi_2}$ y finalmente se realiza el cálculo del producto de las incertidumbres.

También se verifica que se cumpla la desigualdad del principio de incertidubre que establece que el producto de las incertidumbres tiene que ser mayor o igual a el conmutador de $H$ y $L$ ensanduchado con los bra-kets asociados a los estados $\ket{\phi_1}$ y $\ket{\phi_2}$.

In [30]:
# Incertidumbre para phi1
DeltaH2 = simplify(((Dagger(phi1)*H**2 *phi1)[0]) - ((Dagger(phi1)*H*phi1)[0])**2)
DeltaL2 = simplify(((Dagger(phi1)*L**2 *phi1)[0]) - ((Dagger(phi1)*L*phi1)[0])**2)
DeltaHDeltaLphi1 = (sqrt(DeltaH2*DeltaL2)).evalf()
print("La incertidumbre para el estado arbitrario 1 es", DeltaHDeltaLphi1)

# Principio de incertidumbre para phi1
A = (1/2)*((Dagger(phi1)*Commutator(H,L))*phi1).evalf()

# Incertidumbre para phi2
DeltaH2 = simplify((Dagger(phi2)*H**2 *phi2)[0] - ((Dagger(phi2)*H*phi2)[0])**2)
DeltaL2 = simplify((Dagger(phi2)*L**2 *phi2)[0] - ((Dagger(phi2)*L*phi2)[0])**2)
DeltaHDeltaLphi2 = (sqrt(DeltaH2*DeltaL2)).evalf()

print("La incertidumbre para el estado arbitrario 2 es", DeltaHDeltaLphi2)

# Principio de incertidumbre para phi2
B = (1/2)*((Dagger(phi2)*Commutator(H,L)*phi2)).evalf()

print("La incertidumbre asociada al estado phi1 debe ser mayor o igual a:", A[0])
print("La incertidumbre asociada al estado phi2 debe ser mayor o igual a:", B[0])

La incertidumbre para el estado arbitrario 1 es 965.301352190534
La incertidumbre para el estado arbitrario 2 es 1286.89547706178
La incertidumbre asociada al estado phi1 debe ser mayor o igual a: -2.36364252615315e-125
La incertidumbre asociada al estado phi2 debe ser mayor o igual a: 5.90910631538287e-126
